In [ ]:
pip install tensorflow keras numpy matplotlib opencv-python scikit-learn


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator



def load_dataset(data_dir, image_size=(128, 128)):
    """Load the images and their labels from the dataset directory."""
    images = []
    labels = []
    class_names = os.listdir(data_dir)

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for image_file in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_file)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels, class_names


train_dir = 'dataset/train'
test_dir = 'dataset/test'
image_size = (128, 128)

train_images, train_labels, class_names = load_dataset(train_dir, image_size)
test_images, test_labels, _ = load_dataset(test_dir, image_size)


train_images = train_images / 255.0
test_images = test_images / 255.0


datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

datagen.fit(train_images)


model = Sequential()


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(class_names), activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    epochs=15,
                    validation_data=(test_images, test_labels))


test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

def plot_history(history):
    plt.figure(figsize=(12, 4))

    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

     
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.show()

plot_history(history)


model.save('plant_leaf_disease_model.h5')

# 9. Prediction Function for User Interface
def predict_image(image_path):
    """Predict the disease for a given plant leaf image."""
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = np.expand_dims(image, axis=0)  
    image = image / 255.0

    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)
    predicted_label = class_names[predicted_class]

    print(f"Predicted Label: {predicted_label}")
    return predicted_label


predict_image('path_to_new_leaf_image.jpg')


## 